# Описание проекта

Подготовьте прототип модели машинного обучения для «Цифры». Компания разрабатывает решения для эффективной работы промышленных предприятий.

Модель должна предсказать коэффициент восстановления золота из золотосодержащей руды. В вашем распоряжении данные с параметрами добычи и очистки. 

Модель поможет оптимизировать производство, чтобы не запускать предприятие с убыточными характеристиками.

Вам нужно:

1. Подготовить данные;
2. Провести исследовательский анализ данных;
3. Построить и обучить модель.

Чтобы выполнить проект, обращайтесь к библиотекам *pandas*, *matplotlib* и *sklearn.* Вам поможет их документация.

## Импортирование необходимых библиотек


In [286]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error as MAE
# pd.set_option('max_columns', 800) 

# 1. Подготовка данных

In [287]:
test_frame = pd.read_csv("gold_recovery_test.csv")
train_frame = pd.read_csv("gold_recovery_train.csv")
full_frame = pd.read_csv("gold_recovery_full.csv")

In [288]:
# train_frame = train_frame.fillna(1e-100)

In [289]:
train_frame = train_frame.replace(float("-inf"),0)

In [290]:
train_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 87 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   date                                                16860 non-null  object 
 1   final.output.concentrate_ag                         16788 non-null  float64
 2   final.output.concentrate_pb                         16788 non-null  float64
 3   final.output.concentrate_sol                        16490 non-null  float64
 4   final.output.concentrate_au                         16789 non-null  float64
 5   final.output.recovery                               15339 non-null  float64
 6   final.output.tail_ag                                16794 non-null  float64
 7   final.output.tail_pb                                16677 non-null  float64
 8   final.output.tail_sol                               16715 non-null  float64


In [291]:
# def recovery(params,cur_recovery = None):
#     index_params = list(params.dropna(axis = 0).index)
    
#     if cur_recovery is not None:
#         indexes_cur_recovery = list(cur_recovery.dropna(axis = 0).index)
#         indexes = [i for i in index_params if i in indexes_cur_recovery]
#         cur_recovery = cur_recovery[indexes]
#     else:
#         indexes = index_params
        
#     params = params.iloc[indexes]
#     (input_au,
#      output_au,
#      output_tail) = (params.iloc[:,0],
#                      params.iloc[:,1],
#                      params.iloc[:,2])
    
#     recovery_metric = (((output_au) * ((input_au) - (output_tail)))/
#                        (1e-100+((input_au) * ((output_au) - (output_tail))))) * 100
    
#     if cur_recovery is None:
#         return recovery_metric
#     else:
#         assert len(recovery_metric) == len(cur_recovery)
        
#         delta = MAE(cur_recovery,recovery_metric)
#         return delta

In [292]:
# print("MAE ", recovery(train_frame[["rougher.input.feed_ag",
#                                     "rougher.output.concentrate_ag",
#                                     "rougher.output.tail_ag"]],
#                         ))

In [293]:
def apply_recovery(row):
    (input_au,
     output_au,
     output_tail) = (row["rougher.input.feed_au"],
                     row["rougher.output.concentrate_au"],
                     row["rougher.output.tail_au"])
    recovery_metric = (((output_au) * ((input_au) - (output_tail)))/
                       (((input_au) * ((output_au) - (output_tail))))) * 100
    return recovery_metric

In [294]:
test = train_frame.dropna(subset = ["rougher.input.feed_au",
                                    "rougher.output.concentrate_au",
                                    "rougher.output.tail_au",
                                    "rougher.output.recovery"],axis = 0).apply(
                                                                        apply_recovery,axis = 1)

In [296]:
MAE(train_frame.dropna(subset = ["rougher.input.feed_au",
                                    "rougher.output.concentrate_au",
                                    "rougher.output.tail_au",
                                    "rougher.output.recovery"])["rougher.output.recovery"],
                       test)

9.444891926100695e-15

в вычислениях recovery в train присутсвует некоторая неточность

In [168]:
import collections
step = collections.Counter()
type_step = collections.Counter()
data = collections.Counter()
list_columns = [i for i in train_frame.columns if i not in test_frame.columns]
for i in list_columns:
    words = list(map(str,i.split(".")))
    step[words[0]] += 1
    type_step[words[1]] += 1
    data[words[2]] += 1
    

In [180]:
list_columns

['final.output.concentrate_ag',
 'final.output.concentrate_pb',
 'final.output.concentrate_sol',
 'final.output.concentrate_au',
 'final.output.recovery',
 'final.output.tail_ag',
 'final.output.tail_pb',
 'final.output.tail_sol',
 'final.output.tail_au',
 'primary_cleaner.output.concentrate_ag',
 'primary_cleaner.output.concentrate_pb',
 'primary_cleaner.output.concentrate_sol',
 'primary_cleaner.output.concentrate_au',
 'primary_cleaner.output.tail_ag',
 'primary_cleaner.output.tail_pb',
 'primary_cleaner.output.tail_sol',
 'primary_cleaner.output.tail_au',
 'rougher.calculation.sulfate_to_au_concentrate',
 'rougher.calculation.floatbank10_sulfate_to_au_feed',
 'rougher.calculation.floatbank11_sulfate_to_au_feed',
 'rougher.calculation.au_pb_ratio',
 'rougher.output.concentrate_ag',
 'rougher.output.concentrate_pb',
 'rougher.output.concentrate_sol',
 'rougher.output.concentrate_au',
 'rougher.output.recovery',
 'rougher.output.tail_ag',
 'rougher.output.tail_pb',
 'rougher.output.ta

In [178]:
step,"\n",type_step,"\n",data

(Counter({'final': 9,
          'primary_cleaner': 8,
          'rougher': 13,
          'secondary_cleaner': 4}),
 '\n',
 Counter({'output': 30, 'calculation': 4}),
 '\n',
 Counter({'concentrate_ag': 3,
          'concentrate_pb': 3,
          'concentrate_sol': 3,
          'concentrate_au': 3,
          'recovery': 2,
          'tail_ag': 4,
          'tail_pb': 4,
          'tail_sol': 4,
          'tail_au': 4,
          'sulfate_to_au_concentrate': 1,
          'floatbank10_sulfate_to_au_feed': 1,
          'floatbank11_sulfate_to_au_feed': 1,
          'au_pb_ratio': 1}))

Заметим, что из 34 пропущенных признака 30 относятся к параметрам продукта и только 4 к рассчетным характеристикам  

1. 9 пропущенных признаков этапа ___, 8 ___, 9___,4 ____ относятся к параметрам продукта
1. 4 пропущенных признака этапа ___ относятся к рассчетным характеристикам
1. На всех этапах отсутсвует информация о хвостовых частях
1. Во всех этапах, кроме повтороной очистки отсутсвует информация о 
    - Из которых:
        - 
1. 8 

# 2. Анализ данных

# 3. Модель

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке выполнения
- [ ]  Выполнен шаг 1: данные подготовлены
    - [ ]  Проверена формула вычисления эффективности обогащения
    - [ ]  Проанализированы признаки, недоступные в тестовой выборке
    - [ ]  Проведена предобработка данных
- [ ]  Выполнен шаг 2: данные проанализированы
    - [ ]  Исследовано изменение концентрации элементов на каждом этапе
    - [ ]  Проанализированы распределения размеров гранул на обучающей и тестовой выборках
    - [ ]  Исследованы суммарные концентрации
    - [ ]  Проанализированы и обработаны аномальные значения
- [ ]  Выполнен шаг 3: построена модель прогнозирования
    - [ ]  Написана функция для вычисления итогового *sMAPE*
    - [ ]  Обучено и проверено несколько моделей
    - [ ]  Выбрана лучшая модель, её качество проверено на тестовой выборке